# PySpark Taskk 

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun 
import numpy as np
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# RDD

Create an RDD from a list of numbers (1,50) using numpy methods


In [5]:
RDD = sc.parallelize(np.arange(1,50))
print(RDD.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


Find the sum, average, maximum, minimum, and count

In [6]:
print("Sum = " , RDD.sum() , "\nAverage = " , RDD.mean() , "\nMax = " , RDD.max() , "\nMin = " , RDD.min() , "\nCount = " , RDD.count())

Sum =  1225 
Average =  25.0 
Max =  49 
Min =  1 
Count =  49


Count how many numbers are even vs. odd.

In [7]:
Odd = RDD.filter(lambda x:x % 2 != 0)
Even = RDD.filter(lambda x:x % 2 == 0)
print("ODD Numbers:",Odd.count()," EVEN Numbbers:",Even.count())

ODD Numbers: 25  EVEN Numbbers: 24


You have the following data of people info ('Name', 'Age'), answer the following questions

In [8]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [9]:
rdd_people.max(key= lambda x:x[1])

('Khaled', 40)

Compute the average age

In [10]:
ages_rdd = rdd_people.map(lambda x:x[1])
ages_rdd.mean()

31.666666666666668

Group all the names by their age

In [11]:
rdd_people.groupBy(lambda x:x[1]).map(lambda x: (x[0], list(x[1]))).collect()

[(40, [('Khaled', 40)]),
 (25, [('Nada ', 25), ('Nada ', 25)]),
 (35, [('Ahmed', 35), ('Ahmed', 35)]),
 (30, [('Mona', 30)])]

# Text RDD

Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [12]:
rdd_russia = sc.textFile("/data/russia.txt")
rdd_russia.collect()

['Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet',
 'The country is famous for its cold winters and vast landscapes',
 'Russia is a major player in global energy production']

Count the total number of lines.

In [13]:
rdd_russia.count()

8

Count how many lines contain the word "Russia"

In [54]:
rdd_russia.filter(lambda line: "Russia" in line).count()

6

Find the most 5 frequent word in the file.

In [58]:
rdd_russia.flatMap(lambda line: line.split()).map(lambda word : (word,1)).reduceByKey(lambda a,b:a+b).sortBy(lambda x: x[1], ascending=False).take(5)

[('is', 7), ('the', 7), ('Russia', 5), ('in', 5), ('world', 3)]

Tokenize words

In [61]:
print(rdd_russia.flatMap(lambda word:word.split()).collect())

['Russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'Moscow', 'is', 'the', 'capital', 'city', 'of', 'Russia', 'The', 'Russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'Russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'The', 'Trans-Siberian', 'Railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'Russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'The', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production']


Remove stopwords (a, the, is, to, in, of). 

In [64]:
stopwords = ['a', 'the', 'is', 'to', 'in', 'of']
rdd_russia_wihout_stopwords = rdd_russia.flatMap(lambda word:word.split()).filter(lambda word:word not in stopwords)
print(rdd_russia_wihout_stopwords.collect())

['Russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'Moscow', 'capital', 'city', 'Russia', 'The', 'Russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'Russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'The', 'Trans-Siberian', 'Railway', 'longest', 'railway', 'line', 'world', 'Russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'The', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'major', 'player', 'global', 'energy', 'production']


Count the frequency of each word

In [68]:
print(rdd_russia_wihout_stopwords.flatMap(lambda line:line.split()).map(lambda word:(word,1)).collect())

[('Russia', 1), ('largest', 1), ('country', 1), ('world', 1), ('by', 1), ('land', 1), ('area', 1), ('Moscow', 1), ('capital', 1), ('city', 1), ('Russia', 1), ('The', 1), ('Russian', 1), ('language', 1), ('one', 1), ('most', 1), ('widely', 1), ('spoken', 1), ('languages', 1), ('world', 1), ('Russia', 1), ('known', 1), ('for', 1), ('its', 1), ('rich', 1), ('history', 1), ('and', 1), ('culture', 1), ('The', 1), ('Trans-Siberian', 1), ('Railway', 1), ('longest', 1), ('railway', 1), ('line', 1), ('world', 1), ('Russia', 1), ('has', 1), ('strong', 1), ('tradition', 1), ('literature,', 1), ('music', 1), ('and', 1), ('ballet', 1), ('The', 1), ('country', 1), ('famous', 1), ('for', 1), ('its', 1), ('cold', 1), ('winters', 1), ('and', 1), ('vast', 1), ('landscapes', 1), ('Russia', 1), ('major', 1), ('player', 1), ('global', 1), ('energy', 1), ('production', 1)]


# DataFrame

In [69]:
schema = 'id integer, name string, age integer, salary integer' 
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [71]:
df.printSchema()
df.show(2)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [72]:
df['name','salary'].show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [84]:
df.agg(fun.avg('salary')).show()


+-----------+
|avg(salary)|
+-----------+
|     6000.0|
+-----------+



Filter employees older than 28

In [89]:
df.select('*').where('age > 28').show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column and Groubed by name

In [ ]:
df.groupBy('name').count().show()
df.select('name').distinct().count() ##


+------+-----+
|  name|count|
+------+-----+
|   Ali|    1|
|Mariam|    2|
|  Omar|    2|
|  Sara|    1|
+------+-----+



4

Group by a the name column and find average salary

In [101]:
df.groupBy('name').avg('salary').show()

+------+-----------+
|  name|avg(salary)|
+------+-----------+
|   Ali|     4000.0|
|Mariam|     6750.0|
|  Omar|     6750.0|
|  Sara|     5000.0|
+------+-----------+



In [102]:
df1 = spark.read.csv("/data/NullData.csv", header=True, inferSchema=True) #this file in shared folder
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Find the avg sales 

In [110]:
df1.agg(fun.avg('sales')).show()
x = fun.avg(df1['sales'])

+----------+
|avg(sales)|
+----------+
|     400.5|
+----------+



Replace null name with 'Unknown' and sales with the avg sales of the column 

In [112]:
df1.fillna({'Name' : 'Unknown' , 'Sales' : '400.5'}).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|400.5|
|emp2|Unknown|400.5|
|emp3|Unknown|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



# End Of Taskk